In [20]:
! python -m spacy download en_vectors_web_lg

    100% |████████████████████████████████| 661.8MB 9.9MB/s eta 0:00:011   35% |███████████▎                    | 232.5MB 18.7MB/s eta 0:00:23    43% |██████████████                  | 288.1MB 1.4MB/s eta 0:04:34    48% |███████████████▌                | 321.1MB 1.1MB/s eta 0:05:00    49% |████████████████                | 328.7MB 3.5MB/s eta 0:01:34    74% |████████████████████████        | 495.3MB 1.3MB/s eta 0:02:09    78% |█████████████████████████▏      | 520.5MB 1.3MB/s eta 0:01:52    79% |█████████████████████████▍      | 524.4MB 1.6MB/s eta 0:01:24
  Running setup.py install for en-vectors-web-lg ... done

    Linking successful
    /home/jendrik/.local/share/virtualenvs/DLWorkshop-o4GQ4Aql/lib/python3.7/site-packages/en_vectors_web_lg
    -->
    /home/jendrik/.local/share/virtualenvs/DLWorkshop-o4GQ4Aql/lib/python3.7/site-packages/spacy/data/en_vectors_web_lg

    You can now load the model via spacy.load('en_vectors_web_lg')



In [21]:
# From: https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)



In [3]:
import gensim
from gensim.models import word2vec
import os
import traceback
home = os.path.expanduser("~")

model_path = home + '/data/workshop_data/GoogleNews-vectors-negative300.bin.gz'



In [5]:
file_id = '0B7XkCwpI5KDYNlNUTTlSS21pQmM'
try:
    import boto3
    s3 = boto3.resource('s3')
    s3.Bucket('workshopqcondata').download_file('data/GoogleNews-vectors-negative300.bin.gz', model_path)
except:
    if not os.path.exists(model_path):
        download_file_from_google_drive(file_id, model_path)

In [7]:
model = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

In [12]:
model.most_similar(positive=['woman', 'king'], negative=['man'], topn=3)

[('queen', 0.7118192911148071),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431011199951)]

In [17]:
import spacy
from spacy.tokens.token import Token
from numpy import dot
from numpy.linalg import norm

In [18]:
nlp = spacy.load('en_vectors_web_lg')

OSError: [E050] Can't find model 'en_vectors_web_lg'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [19]:
vecs = nlp('king man woman')
out_vec = vecs[0].vector - vecs[1].vector + vecs[2].vector
print(out_vec)

NameError: name 'nlp' is not defined

In [49]:

# cosine similarity
cosine = lambda v1, v2: dot(v1, v2) / (norm(v1) * norm(v2))

def get_most_similar_word(vec, model):
    print(len(model.vocab))
    # gather all known words, take only the lowercased versions
    allWords = list({w for w in model.vocab if w.has_vector})
    
    allWords.sort(key=lambda w: cosine(w.vector, vec))
    allWords.reverse()
    print("Top 10 most similar words:")
    for word in allWords[:10]:
        print(word.orth_, cosine(word.vector, vec))

In [50]:
get_most_similar_word(out_vec, nlp)

1071393


/home/jendrik/.local/share/virtualenvs/dlworkshop-VuPJaqFs/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in float_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


Top 10 most similar words:
fornaro
Post-Teledyne
deferring
THERMAL-PROCESS
LLADRO
sketchers
Otakar
Busacker
Erlandson
kaczorowski
